In [1]:
import torch
from src.models.blm.pl_dataloader import TinyStoriesDataloader
from src.models.blm.pl_training import Transformer

Seed set to 123
Seed set to 123


In [2]:
BASE_URL = "/home/pranav-pc/projects/OpenTransformer/multiformer"
MODEL_CHECKPOINT_PATH = BASE_URL + "/model_checkpoints/blm/last-v3.ckpt"
data_path_train = BASE_URL + "/data/interim/TinyStories_train_65>tk>512.hf"
data_path_val = BASE_URL + "/data/interim/TinyStories_val_65>tk>512.hf"
tokenizer_path = BASE_URL + "/tokenizer_checkpoints"


batch_size = 32
num_workers = 26
ds = TinyStoriesDataloader(data_path_train, data_path_val, tokenizer_path, batch_size, num_workers)

In [3]:
model = Transformer.load_from_checkpoint(MODEL_CHECKPOINT_PATH)
model = torch.compile(model, dynamic=True)

#### Inference
model.eval()
model = model.cuda()

import os

os.environ["WANDB_DISABLED"] = "true"

In [4]:
text = "Lili wanted to watch either a movie or a cartoon. Her mother didn’t let her watch a cartoon so instead she"


# text = (
#     "Lili is a good boy. one day his father called and asked for the school exam result"
# )
def predict(text, max_new_tokens=450, temperature=1.1, top_k=5, conditional_break=[13, 13, 1]):
    tokens = torch.LongTensor(ds.tokenizer.encode_as_ids(text))[:-1].to("cuda:0").view(1, -1)
    # print('tokens',ds.tokenizer.encode(text,out_type=str)[:-1])
    print(
        ds.tokenizer.decode_ids(
            model.predict_step(
                tokens,
                None,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k,
                conditional_break=conditional_break,
            )[0].tolist()
        )
    )


predict(text)

Lili wanted to watch either a movie or a cartoon. Her mother didn’t let her watch a cartoon so instead she gave her a ticket for the movies and told her to be careful. Lili was happy, but she also felt a bit embarrassed when her mother did not let her watch the show. She wished her mother would forgive her and let her watch the movie. So, she decided that she was sorry and that she would never be so embarrassed.




In [5]:
predict("Once upon a time, Sam")

Once upon a time, Sammy was in the kitchen. He was very hungry and wanted to get something. He looked around and saw some food on the table. He was so excited that he ran over and grabbed some food. But then he realized he had taken a long time to get it.

Sammy was feeling very guilty. He knew he shouldn't have taken the food without paying. So he quickly took a nap and when he woke up, he was feeling much better.

The moral of the story is that it's important to be careful and not take things without paying attention.




1. Perplexity
Perplexity is a commonly used measure to evaluate the performance of language models. It quantifies how well the model predicts a sample of text. Lower perplexity 2 values indicate better performance 